# CTF Transformer 训练可视化仪表盘

基于 Panel + HoloViews 的交互式训练监控系统

**功能特性**:
- 深色/浅色主题切换
- 多页签布局
- 自动/手动数据刷新
- 实时训练监控

In [ ]:
# Cell 1: 导入和配置
import panel as pn
import holoviews as hv
import hvplot.pandas
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import param

# 初始化扩展
pn.extension('tabulator', sizing_mode='stretch_width')
hv.extension('bokeh')

# 配置
LOG_DIR = Path('logs')
CHECKPOINT_DIR = Path('checkpoints')
REFRESH_INTERVAL = 5000  # 5秒

print('✓ 依赖加载完成')

In [ ]:
# Cell 2: 数据加载器
class TrainingDataLoader:
    """训练数据加载器 - 支持实时刷新"""
    
    def __init__(self, log_dir: Path = LOG_DIR):
        self.log_dir = log_dir
        self._cache = {}
        self._last_load = None
    
    def get_experiments(self) -> list:
        """获取所有实验名称"""
        if not self.log_dir.exists():
            return []
        return [d.name for d in self.log_dir.iterdir() if d.is_dir()]
    
    def load_training_log(self, experiment: str) -> pd.DataFrame:
        """加载训练日志CSV"""
        csv_path = self.log_dir / experiment / 'training_log.csv'
        if not csv_path.exists():
            return pd.DataFrame()
        
        df = pd.read_csv(csv_path)
        # 解析百分比列
        if 'draw_rate' in df.columns:
            df['draw_rate_pct'] = df['draw_rate'].str.rstrip('%').astype(float)
        return df
    
    def get_latest_stats(self, experiment: str) -> dict:
        """获取最新统计数据"""
        df = self.load_training_log(experiment)
        if df.empty:
            return {}
        
        latest = df.iloc[-1]
        return {
            'generation': int(latest['generation']),
            'temperature': float(latest['temperature']),
            'best_fitness': float(latest['best_fitness']),
            'avg_fitness': float(latest['avg_fitness']),
            'num_games': int(latest['num_games']),
            'draw_rate': latest.get('draw_rate', 'N/A'),
            'sparse_enabled': latest.get('sparse_enabled', False)
        }

# 创建全局数据加载器
data_loader = TrainingDataLoader()
print(f'✓ 数据加载器初始化完成，发现实验: {data_loader.get_experiments()}')

In [ ]:
# Cell 3: 主题配置
DARK_THEME = {
    'background': '#1a1a2e',
    'foreground': '#eaeaea',
    'accent': '#00d4ff',
    'success': '#00ff88',
    'warning': '#ffaa00',
    'error': '#ff4444'
}

LIGHT_THEME = {
    'background': '#ffffff',
    'foreground': '#333333',
    'accent': '#0066cc',
    'success': '#00aa55',
    'warning': '#cc8800',
    'error': '#cc0000'
}

# 主题切换器
theme_switch = pn.widgets.Toggle(name='Dark Mode', value=True, button_type='primary')

def get_theme():
    return DARK_THEME if theme_switch.value else LIGHT_THEME

print('✓ 主题配置完成')

In [ ]:
# Cell 4: 概览面板组件
class OverviewPanel(param.Parameterized):
    """概览面板 - 显示关键指标"""
    
    experiment = param.Selector(default='quick_test', objects=['quick_test'])
    
    def __init__(self, **params):
        super().__init__(**params)
        experiments = data_loader.get_experiments()
        if experiments:
            self.param.experiment.objects = experiments
            self.experiment = experiments[0]
    
    @param.depends('experiment')
    def stats_cards(self):
        """生成统计卡片"""
        stats = data_loader.get_latest_stats(self.experiment)
        if not stats:
            return pn.pane.Markdown('## 暂无数据')
        
        theme = get_theme()
        
        cards = pn.Row(
            pn.indicators.Number(
                name='当前世代', value=stats['generation'],
                format='{value}', font_size='28pt',
                title_size='12pt'
            ),
            pn.indicators.Number(
                name='最佳适应度', value=stats['best_fitness'],
                format='{value:.2f}', font_size='28pt',
                colors=[(0, theme['error']), (50, theme['warning']), (100, theme['success'])]
            ),
            pn.indicators.Number(
                name='平均适应度', value=stats['avg_fitness'],
                format='{value:.2f}', font_size='28pt'
            ),
            pn.indicators.Number(
                name='温度', value=stats['temperature'],
                format='{value:.4f}', font_size='28pt'
            ),
        )
        return cards

overview = OverviewPanel()
print('✓ 概览面板组件创建完成')

In [ ]:
# Cell 5: 适应度演化图
def create_fitness_plot(experiment: str):
    """创建适应度演化图"""
    df = data_loader.load_training_log(experiment)
    if df.empty:
        return hv.Text(0, 0, '暂无数据')
    
    # 创建多线图
    best = hv.Curve(
        df, 'generation', 'best_fitness', label='Best'
    ).opts(color='#00ff88', line_width=2)
    
    avg = hv.Curve(
        df, 'generation', 'avg_fitness', label='Average'
    ).opts(color='#00d4ff', line_width=2)
    
    worst = hv.Curve(
        df, 'generation', 'worst_fitness', label='Worst'
    ).opts(color='#ff4444', line_width=2, line_dash='dashed')
    
    plot = (best * avg * worst).opts(
        title='适应度演化',
        xlabel='世代',
        ylabel='适应度',
        legend_position='top_left',
        height=400,
        responsive=True
    )
    return plot

print('✓ 适应度图表函数创建完成')

In [ ]:
# Cell 6: 温度退火曲线
def create_temperature_plot(experiment: str):
    """创建温度退火曲线"""
    df = data_loader.load_training_log(experiment)
    if df.empty:
        return hv.Text(0, 0, '暂无数据')
    
    plot = hv.Curve(
        df, 'generation', 'temperature'
    ).opts(
        title='温度退火曲线',
        xlabel='世代',
        ylabel='温度',
        color='#ffaa00',
        line_width=2,
        height=300,
        responsive=True
    )
    return plot

print('✓ 温度曲线函数创建完成')

In [ ]:
# Cell 7: 对战统计图
def create_battle_stats_plot(experiment: str):
    """创建对战统计堆叠柱状图"""
    df = data_loader.load_training_log(experiment)
    if df.empty:
        return hv.Text(0, 0, '暂无数据')
    
    # 准备数据
    battle_df = df[['generation', 'l_wins', 'r_wins', 'draws']].melt(
        id_vars='generation',
        var_name='result',
        value_name='count'
    )
    
    colors = {'l_wins': '#00ff88', 'r_wins': '#ff4444', 'draws': '#888888'}
    labels = {'l_wins': 'L胜', 'r_wins': 'R胜', 'draws': '平局'}
    battle_df['label'] = battle_df['result'].map(labels)
    
    plot = hv.Bars(
        battle_df, ['generation', 'label'], 'count'
    ).opts(
        title='对战结果统计',
        xlabel='世代',
        ylabel='场次',
        height=350,
        responsive=True,
        stacked=True
    )
    return plot

print('✓ 对战统计图函数创建完成')

In [ ]:
# Cell 8: 平局率趋势图
def create_draw_rate_plot(experiment: str):
    """创建平局率趋势图"""
    df = data_loader.load_training_log(experiment)
    if df.empty or 'draw_rate_pct' not in df.columns:
        return hv.Text(0, 0, '暂无数据')
    
    # 平局率曲线
    curve = hv.Curve(
        df, 'generation', 'draw_rate_pct'
    ).opts(color='#888888', line_width=2)
    
    # 90%阈值线
    threshold = hv.HLine(90).opts(
        color='#ff4444', line_dash='dashed', line_width=1
    )
    
    plot = (curve * threshold).opts(
        title='平局率趋势 (90%阈值)',
        xlabel='世代',
        ylabel='平局率 (%)',
        height=300,
        responsive=True
    )
    return plot

print('✓ 平局率图函数创建完成')

In [ ]:
# Cell 9: 奖励系统权重可视化
def create_reward_weights_plot():
    """创建奖励系统权重演化图（理论曲线）"""
    gens = np.arange(0, 201)
    dense_weights = []
    sparse_weights = []
    
    for g in gens:
        if g <= 50:
            d, s = 0.8, 0.2
        elif g <= 100:
            progress = (g - 50) / 50
            d = 0.8 - 0.7 * progress
            s = 0.2 + 0.7 * progress
        elif g <= 150:
            d, s = 0.1, 0.9
        else:
            d, s = 0.0, 1.0
        dense_weights.append(d)
        sparse_weights.append(s)
    
    df = pd.DataFrame({
        'generation': gens,
        'Dense': dense_weights,
        'Sparse': sparse_weights
    })
    
    dense = hv.Area(df, 'generation', 'Dense', label='Dense').opts(color='#00d4ff', alpha=0.7)
    sparse = hv.Area(df, 'generation', 'Sparse', label='Sparse').opts(color='#ff8800', alpha=0.7)
    
    plot = (dense * sparse).opts(
        title='奖励权重演化 (课程学习)',
        xlabel='世代',
        ylabel='权重',
        height=300,
        responsive=True
    )
    return plot

print('✓ 奖励系统图函数创建完成')

In [ ]:
# Cell 10: 数据表格
def create_data_table(experiment: str):
    """创建训练数据表格"""
    df = data_loader.load_training_log(experiment)
    if df.empty:
        return pn.pane.Markdown('暂无数据')
    
    # 选择显示的列
    display_cols = [
        'generation', 'temperature', 'best_fitness', 
        'avg_fitness', 'l_wins', 'r_wins', 'draws', 'draw_rate'
    ]
    display_df = df[[c for c in display_cols if c in df.columns]]
    
    table = pn.widgets.Tabulator(
        display_df,
        pagination='remote',
        page_size=15,
        sizing_mode='stretch_width'
    )
    return table

print('✓ 数据表格函数创建完成')

In [ ]:
# Cell 11: 刷新控制
refresh_button = pn.widgets.Button(name='刷新数据', button_type='primary')
auto_refresh = pn.widgets.Toggle(name='自动刷新 (5s)', value=False)

# 实验选择器
experiment_select = pn.widgets.Select(
    name='选择实验',
    options=data_loader.get_experiments() or ['quick_test'],
    value=data_loader.get_experiments()[0] if data_loader.get_experiments() else 'quick_test'
)

print('✓ 控制组件创建完成')

In [ ]:
# Cell 12: 构建页签内容
def build_overview_tab():
    """概览页签"""
    return pn.Column(
        pn.pane.Markdown('## 训练概览'),
        pn.bind(lambda e: overview.stats_cards(), experiment_select),
        pn.bind(create_fitness_plot, experiment_select),
        sizing_mode='stretch_width'
    )

def build_fitness_tab():
    """适应度页签"""
    return pn.Column(
        pn.pane.Markdown('## 适应度分析'),
        pn.bind(create_fitness_plot, experiment_select),
        pn.bind(create_temperature_plot, experiment_select),
        sizing_mode='stretch_width'
    )

print('✓ 页签构建函数创建完成 (1/2)')

In [ ]:
# Cell 13: 更多页签构建
def build_battle_tab():
    """对战统计页签"""
    return pn.Column(
        pn.pane.Markdown('## 对战统计'),
        pn.bind(create_battle_stats_plot, experiment_select),
        pn.bind(create_draw_rate_plot, experiment_select),
        sizing_mode='stretch_width'
    )

def build_reward_tab():
    """奖励系统页签"""
    return pn.Column(
        pn.pane.Markdown('## 奖励系统'),
        create_reward_weights_plot(),
        sizing_mode='stretch_width'
    )

def build_data_tab():
    """数据页签"""
    return pn.Column(
        pn.pane.Markdown('## 训练数据'),
        pn.bind(create_data_table, experiment_select),
        sizing_mode='stretch_width'
    )

print('✓ 页签构建函数创建完成 (2/2)')

In [ ]:
# Cell 14: 组装完整仪表盘
# 顶部控制栏
header = pn.Row(
    pn.pane.Markdown('# CTF Transformer 训练监控'),
    pn.Spacer(),
    experiment_select,
    theme_switch,
    refresh_button,
    auto_refresh,
    sizing_mode='stretch_width'
)

# 多页签布局
tabs = pn.Tabs(
    ('概览', build_overview_tab()),
    ('适应度', build_fitness_tab()),
    ('对战统计', build_battle_tab()),
    ('奖励系统', build_reward_tab()),
    ('数据表', build_data_tab()),
    dynamic=True
)

print('✓ 仪表盘组装完成')

In [ ]:
# Cell 15: 显示仪表盘
dashboard = pn.Column(
    header,
    pn.layout.Divider(),
    tabs,
    sizing_mode='stretch_width'
)

# 显示仪表盘
dashboard.servable()
dashboard